In [102]:
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.graph_objects as go

import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

import time

2.6.0+cu126
12.6
90501
CUDA available: True
GPU: NVIDIA GeForce RTX 4070 SUPER


In [103]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Path to the .env file inside 01 - Documentation
env_path = Path("01 - Documentation") / "keys.env"

# Load the .env file from that path
load_dotenv(dotenv_path=env_path)

# Now you can access your keys
API_KEY = os.getenv("ALPACA_API_KEY")
SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")

print(API_KEY[:4] + "****")  # optional: verify it's working


PKD5****


### Stock Pick and Time Frame Choice


In [104]:
eq_symbol = "SPY"
startYear = 2020
endYear = 2023

In [105]:
# Create data client
client = StockHistoricalDataClient(API_KEY, SECRET_KEY)

# Define request
request_params = StockBarsRequest(
    symbol_or_symbols=[eq_symbol],
    timeframe=TimeFrame.Day,
    start=datetime(startYear, 1, 1),
    end=datetime(endYear, 12, 31)
)

# Fetch data
bars = client.get_stock_bars(request_params)
stockData = bars.df
stockData = stockData.reset_index()


print(stockData.head())
print(stockData.columns)
#print(df.index)
print(stockData.shape)


  symbol                 timestamp    open    high     low   close  \
0    SPY 2020-01-02 05:00:00+00:00  323.54  324.89  322.53  324.87   
1    SPY 2020-01-03 05:00:00+00:00  321.16  323.64  321.10  322.43   
2    SPY 2020-01-06 05:00:00+00:00  320.49  323.73  320.36  323.73   
3    SPY 2020-01-07 05:00:00+00:00  323.02  323.54  322.24  322.74   
4    SPY 2020-01-08 05:00:00+00:00  322.94  325.78  322.67  324.42   

       volume  trade_count        vwap  
0  60187033.0     304886.0  323.680084  
1  80319689.0     358026.0  322.732865  
2  56672077.0     255769.0  322.602237  
3  43646563.0     226060.0  322.918261  
4  69691471.0     340005.0  324.553163  
Index(['symbol', 'timestamp', 'open', 'high', 'low', 'close', 'volume',
       'trade_count', 'vwap'],
      dtype='object')
(1006, 9)


In [106]:
import plotly.express as px

# Plot AAPL closing price over time
fig = px.line(stockData, x='timestamp', y='close', title = eq_symbol + ' Closing Prices (' + str(startYear) + '-' + str(endYear) + ')')
fig.update_layout(xaxis_title='Date', yaxis_title='Close Price (USD)', width=1000, height=600)
fig.show()


### Moving Average on the data

Analyzing the price signal with a moving average calculated over 3 interesting time spans.

5 days, 10 days, 20 days

$$
y_t = \mu + \sum_{j=1}^{q} \theta_j \epsilon_{t-j} + \epsilon_t
$$

\begin{aligned}
\textbf{where:} \\
y_t &= \text{value at time } t \\
\mu &= \text{mean of the series} \\
\theta_j &= \text{MA coefficient for lag } j \\
\epsilon_{t-j} &= \text{white noise error at lag } j \\
\epsilon_t &= \text{current white noise error}
\end{aligned}



In [107]:
# Calculate moving averages
start = time.time()
movingAvg5day = stockData['close'].rolling(window=5).mean()
ma5_time = time.time() - start

start = time.time()
movingAvg10day = stockData['close'].rolling(window=10).mean()
ma10_time = time.time() - start

start = time.time()
movingAvg20day = stockData['close'].rolling(window=20).mean()
ma20_time = time.time() - start


# Create the figure
fig = go.Figure()
# stockData
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=stockData['close'],
                         mode='lines', name='Close'))
# Add the 10-day moving average
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=movingAvg10day,
                         mode='lines', name='10-Day SMA'))
# Add the 20-day moving average
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=movingAvg20day,
                         mode='lines', name='20-Day SMA'))
# Add the 5-day moving average
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=movingAvg5day,
                         mode='lines', name='5-Day SMA'))

fig.update_layout(title= eq_symbol + ' Closing Price with 10-Day Moving Average' + ' (' + str(startYear) + '-' + str(endYear) + ')',
                  xaxis_title='Date', yaxis_title='Price (USD)', width=1000,
                   height=600)

# Show the plot
fig.show()


### Auto Regressive Model Fit - scikit

Takes the last 5 trading days and fits a linear regression to the data.

$$
y_t = c + \sum_{i=1}^{p} \phi_i y_{t-i} + \epsilon_t
$$

\begin{aligned}
\textbf{where:} \\
y_t &= \text{value at time } t \\
c &= \text{constant (intercept term)} \\
\phi_i &= \text{AR coefficient for lag } i \\
y_{t-i} &= \text{value at lag } i \\
\epsilon_t &= \text{white noise error term at time } t
\end{aligned}




In [108]:
from sklearn.linear_model import LinearRegression

# Extract the 'close' series and drop any NaNs
close_series = stockData['close'].dropna()
close_array = close_series.values

# Lag setup
lag = 5
X, y = [], []
for i in range(lag, len(close_array)):
    X.append(close_array[i-lag:i])
    y.append(close_array[i])

X, y = np.array(X), np.array(y)

start = time.time()
# Fit model
model = LinearRegression().fit(X, y)
ARpreds = model.predict(X)
ar_time = time.time() - start

fig = go.Figure()
# stockData
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=stockData['close'],
                         mode='lines', name='Close'))
# Add the AR predictions
fig.add_trace(go.Scatter(x=stockData['timestamp'][lag:], y=ARpreds,
                         mode='lines', name='AR Predictions'))
fig.update_layout(title= eq_symbol + ' Closing Prices vs AR(5) Model (Scikit-Learn)' + ' (' + str(startYear) + '-' + str(endYear) + ')',
                  xaxis_title='Date', yaxis_title='Price (USD)', width=1000,
                   height=600)
fig.show()

### ARIMA Model - Auto Regressive Integrated Moving Average

The **ARIMA (AutoRegressive Integrated Moving Average)** model is a powerful time series forecasting method that combines autoregressive terms, differencing to remove trends, and moving averages of past errors. It is well-suited for modeling non-stationary data with underlying patterns but no clear seasonality.

$$
\Delta^d y_t = c + \sum_{i=1}^{p} \phi_i \Delta^d y_{t-i} + \sum_{j=1}^{q} \theta_j \epsilon_{t-j} + \epsilon_t
$$

\begin{aligned}
\textbf{where:} \\
\Delta^d y_t &= \text{the } d\text{-th order differenced series} \\
c &= \text{constant term} \\
\phi_i &= \text{AR coefficient} \\
\theta_j &= \text{MA coefficient} \\
\epsilon_t &= \text{white noise error at time } t \\
p &= \text{number of autoregressive lags} \\
d &= \text{number of differences (to make the series stationary)} \\
q &= \text{number of moving average lags}
\end{aligned}



In [113]:
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima

# Use clean close price series
# close_series = stockData['close'].dropna()

# Fit an ARIMA(p=5, d=1, q=0) model as a starting point
# model = ARIMA(close_series, order=(5, 1, 0))  # ARIMA(5,1,0) ≈ AR(5) on differenced data

# model_fit = ARIMA_model.fit()

# In-sample predictions
# ARIMA_preds = model_fit.predict(start=5, end=len(close_series)-1, typ='levels')

# Automatically find the best ARIMA(p,d,q)
start = time.time()
ARIMA_model = auto_arima(close_series,
                         seasonal=False,
                         stepwise=True,
                         suppress_warnings=True,
                         trace=True)

ARIMA_preds = ARIMA_model.predict_in_sample()
ARIMA_preds = ARIMA_preds[1:]  # Adjust for the lag
arima_time = time.time() - start

fig = go.Figure()
# stockData
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=stockData['close'],
                         mode='lines', name='Close'))
fig.add_trace(go.Scatter(
    x=stockData['timestamp'][-len(ARIMA_preds):], 
    y=ARIMA_preds, 
    name="ARIMA(5,1,0)"
))
fig.update_layout(title= eq_symbol + ' Close vs ARIMA Model' + ' (' + str(startYear) + '-' + str(endYear) + ')',
                  xaxis_title='Date', yaxis_title='Price (USD)', width=1000,
                   height=600)
fig.show()


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=6006.807, Time=0.56 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=6046.913, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=6042.287, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=6042.989, Time=0.10 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=6045.853, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=6042.452, Time=0.20 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=6042.740, Time=0.25 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=6007.255, Time=0.57 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=6045.337, Time=0.66 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=6041.963, Time=0.16 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=6043.342, Time=0.34 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=6043.783, Time=0.26 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=6010.279, Time=0.98 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=6005.883, Time=0.34 sec
 ARIMA(1,1,2)(0,0,0)[0]          

### SARIMA Model - Seasonal ARIMA

The **SARIMA (Seasonal AutoRegressive Integrated Moving Average)** model extends ARIMA by incorporating **seasonal patterns** in time series data, making it ideal for datasets with recurring trends (like weekly or monthly cycles). It combines non-seasonal ARIMA terms with seasonal ARIMA components to capture both short-term dynamics and long-term seasonal effects.

$$
\Phi_P(L^s)\phi_p(L)(1 - L)^d(1 - L^s)^D y_t = \Theta_Q(L^s)\theta_q(L) \epsilon_t
$$

**Where:**

$$
\begin{aligned}
y_t & = \text{observed value at time } t \\
L & = \text{lag operator (e.g., } L y_t = y_{t-1}) \\
\phi_p(L) & = \text{non-seasonal AR (autoregressive) polynomial of order } p \\
\theta_q(L) & = \text{non-seasonal MA (moving average) polynomial of order } q \\
\Phi_P(L^s) & = \text{seasonal AR polynomial of order } P \text{ with period } s \\
\Theta_Q(L^s) & = \text{seasonal MA polynomial of order } Q \text{ with period } s \\
d & = \text{non-seasonal differencing order} \\
D & = \text{seasonal differencing order} \\
s & = \text{seasonal period (e.g., 5 for weekly seasonality in daily data)} \\
\epsilon_t & = \text{white noise error term}
\end{aligned}
$$


In [114]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima

# model = SARIMAX(close_series,
#                 order=(1, 1, 1),
#                 seasonal_order=(1, 1, 1, 5),
#                 enforce_stationarity=False,
#                 enforce_invertibility=False)

# sarima_result = model.fit()

# sarima_preds = sarima_result.predict(start=10, end=len(close_series)-1, dynamic=False)

# Automatically find the best SARIMA model
start = time.time()
sarima_model = auto_arima(close_series,
                          seasonal=True,
                          m=5,  # seasonality period (e.g., 5 trading days per week)
                          stepwise=True,
                          suppress_warnings=True,
                          trace=True)

SARIMA_preds = sarima_model.predict_in_sample()
SARIMA_preds = SARIMA_preds[1:]
sarima_time = time.time() - start

fig = go.Figure()
# stockData
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=stockData['close'],
                         mode='lines', name='Close'))
fig.add_trace(go.Scatter(
    x=stockData['timestamp'][-len(SARIMA_preds):], 
    y=SARIMA_preds, 
    name="SARIMA(1,1,1)x(1,1,1,5)"
))
fig.update_layout(title= eq_symbol + ' Closing Prices vs SARIMA Model' + ' (' + str(startYear) + '-' + str(endYear) + ')',
                  xaxis_title='Date', yaxis_title='Price (USD)', width=1000,
                   height=600)
fig.show()

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[5] intercept   : AIC=inf, Time=1.48 sec
 ARIMA(0,1,0)(0,0,0)[5] intercept   : AIC=6046.913, Time=0.02 sec
 ARIMA(1,1,0)(1,0,0)[5] intercept   : AIC=6043.155, Time=0.17 sec
 ARIMA(0,1,1)(0,0,1)[5] intercept   : AIC=6043.633, Time=0.17 sec
 ARIMA(0,1,0)(0,0,0)[5]             : AIC=6045.853, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[5] intercept   : AIC=6042.287, Time=0.05 sec
 ARIMA(1,1,0)(0,0,1)[5] intercept   : AIC=6043.043, Time=0.12 sec
 ARIMA(1,1,0)(1,0,1)[5] intercept   : AIC=6044.712, Time=0.52 sec
 ARIMA(2,1,0)(0,0,0)[5] intercept   : AIC=6040.888, Time=0.07 sec
 ARIMA(2,1,0)(1,0,0)[5] intercept   : AIC=6041.986, Time=0.24 sec
 ARIMA(2,1,0)(0,0,1)[5] intercept   : AIC=6041.904, Time=0.19 sec
 ARIMA(2,1,0)(1,0,1)[5] intercept   : AIC=6043.640, Time=0.37 sec
 ARIMA(3,1,0)(0,0,0)[5] intercept   : AIC=6042.478, Time=0.12 sec
 ARIMA(2,1,1)(0,0,0)[5] intercept   : AIC=6042.740, Time=0.24 sec
 ARIMA(1,1,1)(0,0,0)[5] intercept   : 

### Combined Modeling

In [115]:
fig = go.Figure()
# stockData
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=stockData['close'],
                         mode='lines', name='Close'))

# Add the 5-day moving average
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=movingAvg5day,
                         mode='lines', name='5-Day SMA'))
# Add the 10-day moving average
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=movingAvg10day,
                         mode='lines', name='10-Day SMA'))
# Add the 20-day moving average
fig.add_trace(go.Scatter(x=stockData['timestamp'], y=movingAvg20day,
                         mode='lines', name='20-Day SMA'))

fig.add_trace(go.Scatter(
    x=stockData['timestamp'][lag - 1 : -1],  # shift back by one
    y=ARpreds,
    mode='lines',
    name='AR Predictions'
))


# fig.add_trace(go.Scatter(x=stockData['timestamp'][5:], y=ARIMA_preds,
#                          mode='lines', name='ARIMA(5,1,0)'))
fig.add_trace(go.Scatter(
    x=stockData['timestamp'][-len(ARIMA_preds) - 1 : -1],
    y=ARIMA_preds,
    mode='lines',
    name='ARIMA(5,1,0)'
))


# fig.add_trace(go.Scatter(x=stockData['timestamp'][10:], y=SARIMA_preds,
#                          mode='lines', name='SARIMA(1,1,1)x(1,1,1,5)'))
fig.add_trace(go.Scatter(
    x=stockData['timestamp'][-len(SARIMA_preds) - 1 : -1],
    y=SARIMA_preds,
    mode='lines',
    name='SARIMA(1,1,1)x(1,1,1,5)'
))


fig.update_layout(title= eq_symbol + ' Closing Prices vs Linear Models' + ' (' + str(startYear) + '-' + str(endYear) + ')',
                  xaxis_title='Date', yaxis_title='Price (USD)', width=1000,
                   height=600)
fig.show()

### Model Evaluation: Error Metrics & Compute Time

In this section, we evaluated six models for predicting stock closing prices:

- **Three Moving Averages**: MA_5, MA_10, MA_20
- **Three Time Series Models**: AR, ARIMA, SARIMA

For each model, we computed the following:

- **R²**: Measures how well the predictions track the true data (closer to 1 is better).
- **MSE**: Mean Squared Error quantifies the average squared difference between predicted and true values.
- **AIC**: Akaike Information Criterion (for ARIMA/SARIMA), used to compare model fit and complexity.
- **Compute Time**: Measured time (in seconds) to fit and generate predictions for each model.

Key insights:
- ARIMA and SARIMA achieved the **best R² and lowest MSE**, with SARIMA being slower to compute.
- AR model had **nearly identical accuracy to ARIMA** but ran **significantly faster**.
- MA_5 offered fast, simple modeling with reasonably good performance.


In [112]:
from sklearn.metrics import r2_score, mean_squared_error

# Trim actual series to match the length of each prediction
actual = close_series.reset_index(drop=True)

metrics = {}

# Moving Averages (skip NaNs at beginning)
for label, preds in {
    "MA_5": movingAvg5day,
    "MA_10": movingAvg10day,
    "MA_20": movingAvg20day
}.items():
    valid = ~preds.isna()
    y_true, y_pred = actual[valid], preds[valid]
    metrics[label] = {
        "AIC": None,  # Not applicable
        "R2": r2_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred)
    }

# AR model
metrics["AR"] = {
    "AIC": None,  # AIC only available if you used statsmodels
    "R2": r2_score(actual[5:], ARpreds),
    "MSE": mean_squared_error(actual[5:], ARpreds)
}

# ARIMA model
metrics["ARIMA"] = {
    "AIC": ARIMA_model.aic(),
    "R2": r2_score(actual[1:], ARIMA_preds),
    "MSE": mean_squared_error(actual[1:], ARIMA_preds)
}

# SARIMA model
metrics["SARIMA"] = {
    "AIC": sarima_model.aic(),
    "R2": r2_score(actual[1:], SARIMA_preds),
    "MSE": mean_squared_error(actual[1:], SARIMA_preds)
}

metrics["MA_5"]["Time (s)"] = ma5_time
metrics["MA_10"]["Time (s)"] = ma10_time
metrics["MA_20"]["Time (s)"] = ma20_time
metrics["AR"]["Time (s)"] = ar_time
metrics["ARIMA"]["Time (s)"] = arima_time
metrics["SARIMA"]["Time (s)"] = sarima_time

metrics_df = pd.DataFrame(metrics).T
metrics_df = metrics_df[["AIC", "R2", "MSE", "Time (s)"]]
metrics_df


,AIC,R2,MSE,Time (s)
MA_5,NaN,0.990082,26.900912,0.000000
MA_10,NaN,0.977074,61.933115,0.001000
MA_20,NaN,0.948621,137.813472,0.000000
AR,NaN,0.991263,23.674658,0.001002
ARIMA,6005.882528,0.991608,22.826360,6.397368
SARIMA,6039.872690,0.991283,23.711630,5.282358
